In [ ]:
pip install tensorflow keras music21 numpy matplotlib pretty_midi


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 26.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 2.4 MB/s eta 0:00:00
  Created wheel for pretty_midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592287 sha256=a3791b67e67bb3ea7c9ee8e3a6589ecb9445486508b08e448a4ff9c3801e622a
  Stored in directory: /root/.cache/pip/wheels/e6/95/ac/15ceaeb2823b04d8e638fd1495357adb8d26c00ccac9d7782e
Successfully built pretty_midi


In [ ]:
from music21 import converter, instrument, note, chord
import glob

def extract_notes():
    notes = []
    for file in glob.glob("midi_songs/*.mid"):  # Path to your MIDI files
        midi = converter.parse(file)

        for part in midi.parts:
            for element in part.flat.notes:
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch))  # Capture pitch
                elif isinstance(element, chord.Chord):
                    notes.append('.'.join(str(n) for n in element.normalOrder))  # Capture chords

    return notes

notes = extract_notes()
print(f"Extracted {len(notes)} notes/chords from MIDI files.")


Extracted 0 notes/chords from MIDI files.


In [ ]:
import numpy as np

# Create a mapping of unique notes to integers
unique_notes = sorted(set(notes))
note_to_int = {note: number for number, note in enumerate(unique_notes)}

# Convert notes to sequences
sequence_length = 50  # Number of notes per training sample
network_input = []
network_output = []

for i in range(len(notes) - sequence_length):
    input_seq = notes[i:i + sequence_length]
    output_note = notes[i + sequence_length]

    network_input.append([note_to_int[n] for n in input_seq])
    network_output.append(note_to_int[output_note])

# Reshape for LSTM input
X = np.reshape(network_input, (len(network_input), sequence_length, 1))
X = X / float(len(unique_notes))  # Normalize
y = np.array(network_output)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Activation
from tensorflow.keras.utils import to_categorical

# Convert y to categorical
y = np.array(network_output, dtype=np.int32)  # Ensure integer format

print(f"X shape: {X.shape}")
print(f"y shape before to_categorical: {len(y)}")
print(f"Unique Notes: {len(unique_notes)}")



model = Sequential([
    LSTM(256, return_sequences=True, input_shape=(X.shape[1], 1)),
    Dropout(0.3),
    LSTM(256, return_sequences=False),
    Dropout(0.3),
    Dense(256, activation='relu'),
    Dense(len(unique_notes), activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()


X shape: (0, 50, 1)
y shape before to_categorical: 0
Unique Notes: 0


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 50, 256)             │         264,192 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 50, 256)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 256)                 │         525,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │          65,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 0)                   │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 855,296 (3.26 MB)

 Trainable params: 855,296 (3.26 MB)

 Non-trainable params: 0 (0.00 B)